In [1]:
import os
import requests
from bs4 import BeautifulSoup

In [3]:
seznam_url = "https://gcatholic.org/documents/data/type-PAC"

In [5]:
seznam_stranka = requests.get(seznam_url)

In [7]:
seznam_polivka = BeautifulSoup(seznam_stranka.text)

In [9]:
dokumenty = []
for radek in seznam_polivka.find_all('tr'):
    if 'Pope' in radek.text:
        dokument = {}
        if radek.find(class_='doc'):
            dokument['nazev'] = radek.find(class_='doc').text.strip()
            dokument['dokument'] = 'konstituce'
            tagy = []
            for t in radek.find_all(class_='tag'):
                tagy.append(t.text.strip())
            dokument['tagy'] = tagy
            dokument['papez'] = radek.find_all('td')[1].text.strip()
            datum = radek.find_all('td')[2].text.strip()
            dokument['datum'] = datum
            dokumenty.append(dokument)

In [11]:
import polars as pl

In [13]:
df = pl.DataFrame(dokumenty).with_columns(pl.col('datum').str.to_date())

In [17]:
df

nazev,dokument,tagy,papez,datum
str,str,list[str],str,date
"""In ecclesiarum communione""","""konstituce""",[],"""Pope Francis""",2023-01-06
"""Prædicate evangelium""","""konstituce""","[""canon law"", ""Roman Curia""]","""Pope Francis""",2022-03-19
"""Pascite gregem Dei""","""konstituce""","[""canon law""]","""Pope Francis""",2021-05-23
"""Episcopalis communio""","""konstituce""","[""synod""]","""Pope Francis""",2018-09-15
"""Veritatis Gaudium""","""konstituce""","[""university""]","""Pope Francis""",2017-12-08
…,…,…,…,…
"""Postquam""","""konstituce""","[""cardinal""]","""Pope Sixtus V""",1586-12-03
"""Quo Primum""","""konstituce""","[""Mass""]","""Saint Pope Pius V""",1570-07-14
"""Alias Nos""","""konstituce""","[""Roman Curia""]","""Pope Pius IV""",1564-08-02


In [15]:
df.write_parquet(os.path.join('data','konstituce.parquet'), use_pyarrow=True)